In [ ]:
# Initializing for the use of Earth Engine services
# In case of the first time use you should authenticate by pasting token in the box and then pressing enter
import ee
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

# The tool to visualize lava flows, burnfires with Sentinel-2 multispectral imagery
The tool allows to enhance the visualization of "high-temperature areas" within the different background: from Natural Colors to different combintations of SWIR bands. Also, the possibility to blend several band visualizations is supported. The choice of all parameters (sensitivity, saturation, manual adjustment, stretching) could be done manually. The full description of all functionality with step-by-step tutorial could be found on github repositorium. Finally, the visualization of final image could be corrected.

In [ ]:
import geemap
import os
import functools
import itertools
import ipywidgets as widgets
from ipyfilechooser import FileChooser
import ipyleaflet
from ipyleaflet import (WidgetControl, Map)
import datetime
import numpy as np
import colorama
from colorama import Fore

---
* **Select the area of interest**

Choose the area of interest on the map or indicate the coordinates and buffer range (the circle-shape area is not supported, select between rectangle or polygon)
In order to define your own area of interest click box `User-defined AOI` and then select the on the map with embedded instruments on the left panel

In [ ]:
# initial assignment of style parameter for the widget: not to cut widget description and show the full name
style = {'description_width': 'initial'}

# widgets set up for coordinates and corresponding buffer and checkbox to select the AOI manually on the map
lat_widget = widgets.FloatText(description='Latitude:', max_height=20,  style=style, layout=widgets.Layout(width='25%'))
long_widget = widgets.FloatText(description='Longtitude:', max_height=20,  style=style, layout=widgets.Layout(width='25%'))
aoi_widget = widgets.Checkbox(value=False, description='User-defined AOI', disabled=False)
buffer_widget = widgets.IntText(description = 'Distance:', layout=widgets.Layout(width='35%'))
zoom_widget = widgets.BoundedIntText(description = 'Zoom:', value=13, min=0, max=24, step=1, layout=widgets.Layout(width='15%'))

# This is the event handler that is called when aoi_widget is checked and unchecked
def aoi_change(change):
    m.layers = m.layers[:3] #deleting layers and keeping only basemaps
    m.user_roi = None #erasing user drawn areas of interest
    m.user_rois = None
    m.draw_count = 0
    lat_widget.value = 0 #setting widgets to default values
    long_widget.value = 0
    buffer_widget.value = 0

# event listener to call function when aoi_widget checkbox is clicked or unclicked
aoi_widget.observe(aoi_change, names='value')

# widgets layout creation through horizontal and vertical boxes, and displaying
coord_widgets = widgets.HBox([lat_widget, long_widget, aoi_widget])
coord_buffer = widgets.VBox([coord_widgets, widgets.HBox([buffer_widget, zoom_widget])])
display(coord_buffer)

* **Select the date range**

After the area of interest has been chosen the next step is to define time interval. Choose the range of dates that you are interested in and then click the button `List images`. After a while a list with all available dates and associated images will appear under these widgets. In case you want to change parameters you could just fill new values to the fields with coordinates or just click `Clear fields`

In [ ]:
# Button widgets for listing images and clearing fields to default values
submit = widgets.Button(description='List images', button_style='primary', tooltip='Click to submit', style=style)
clear_fields = widgets.Button(description='Clear fields', button_style='warning', tooltip='Click to clear', style=style)

# Widgets to select the date of start and end
start_day = widgets.DatePicker(description='Start date: ', disabled=False)
end_day = widgets.DatePicker(description='End date: ', disabled=False)

# Output widget field under the buttons, hidden, and will print some text after when we will call it
output_widget_listimages = widgets.Output()

# Widgets layout and dislpaying
widget_buttons = widgets.HBox([submit, clear_fields])
widget_dates = widgets.HBox([start_day, end_day])
final_widget = widgets.VBox([widget_dates, widget_buttons, output_widget_listimages])
display(final_widget)

* **Select the image**

After the date list is appeared on your screen you should choose one of your interest for further visualization. After that type its number to the field below and click `Submit`. If everything is correct the message *"The image is selected"* will appear on the screen. 

In [ ]:
# Widget to select the date from the list above, and widget to submit the choice
day_select_widget = widgets.IntText(value=1, description='Date no.:', disabled=False)
submit_sel = widgets.Button(description='Submit', button_style='primary', tooltip='Click to submit', style=style)

# Output widget that will confirm that selection is succesful
output_widget_img = widgets.Output()

# Widgets layout and displaying
display(widgets.HBox([day_select_widget, submit_sel]))
display(output_widget_img)

In [ ]:
# Map visualization, adding Google Terrain basemap
m = geemap.Map()
m.add_basemap('TERRAIN')

# Layer control is a panel to turn on/off layers provided by ipyleaflet
m.addLayerControl()

# Output widget with output control allows to display output on a map when called
output_widget = widgets.Output()
output_control = WidgetControl(widget=output_widget, position='bottomright')
m.add_control(output_control)

# Map diplaying
m

In [ ]:
# Event handler when clicked on the button to list images for the period of interest
def on_list_clicked(b):
    # The output widget under the button to show that processing is ongoing and after list all dates with images
    with output_widget_listimages:
        # Clear if if smth is present there and print 
        output_widget_listimages.clear_output()
        print("Listing images...")
        
        try:
            # assign true/false value from the checkbox of user-defined AOI
            use_aoi = aoi_widget.value
            
            #if the checkbox id True 
            if use_aoi:
                if m.user_roi is not None: # if user defined drawings assign them to roi and create a centroid point
                    global roi
                    roi = m.user_roi
                    layer_name = 'User drawn AOI'
                    point = roi.centroid(maxError = 2)
                else:
                    # if there are no user drawings print the output
                    output_widget_listimages.clear_output()
                    print('No user AOI is defined')
                    return
            # second scenario is from cooridnates and buffer
            else:
                # assign values from vidgets
                x_coords = long_widget.value
                y_coords = lat_widget.value
                buffer_dist = buffer_widget.value
                #create a point with a buffer and find its bounding box to obtain "square" of interest
                point = ee.Geometry.Point([x_coords, y_coords])
                buffer = point.buffer(int(buffer_dist))
                roi = buffer.bounds()
                layer_name = 'AOI from coordinates'
            
            # add layer that show the area of interest, assigned depending on the prevously processed option
            m.addLayer(ee.Image().paint(roi, 0, 2), {'palette': 'red'}, layer_name)
            
            # Assign start end end values to variables from widgets in a specific format for further tranformation to ee.Date
            start = ee.Date(start_day.value.strftime('%Y-%m-%d'))
            end = ee.Date(end_day.value.strftime('%Y-%m-%d'))
            
            # Centering the map on the area of interest
            zoom_level = zoom_widget.value
            m.centerObject(point, zoom_level)
            
            # Retrieval of the collection, see chunk 12, collection is globally defined to be accesed openly as a varible outside the function
            global collection
            collection = collection_retr(start, end, roi)
            imagelist = collection.toList(collection.size()) #do we need image date??
            
            # accesing all image dates in a ee format
            allDates = ee.List(collection.aggregate_array('system:time_start')).getInfo();
            
            #fucntion to obtain date of image in defined format
            def date_retr(date):
                return ee.Date(date).format('YYYY-MM-dd').getInfo()
            
            # retrieving all dates in a simple format through map iterative fucntion and converting to numpy array
            allDatesSimple = list((map(date_retr, allDates)))
            allDatesSimple = np.array(allDatesSimple)
            
            # clearing the widget to list the dates there
            output_widget_listimages.clear_output()
            # global variable to be accessed from other fucntions
            global list_dates
            # select only unique dates for the image (because often images are listed twice for the same day)
            list_dates = np.unique(allDatesSimple)
            # loop through the array to print the unique days
            # +1 in order to convert from pyton numeration that starts from 0
            for num, date in enumerate(list_dates):
                print(str(num+1) + ') ' + date)
                
        except Exception as e:
            #print the exception in case of occured problems
            print(e)
            print(Fore.RED + 'Try to select other parameters. An error occured!')

            
# Event listener with the button list images clicked               
submit.on_click(on_list_clicked)

# fucntion to clear fields (widgets) and put them to default
def clear_fields_clicked(b):
    try:
        # set all widgets to default and clear map layers, keeping only baselayers
        lat_widget.value = 0
        m.layers = m.layers[:3]  
        long_widget.value = 0 
        aoi_widget.value = False
        buffer_widget.value = 0
        start_day.value = None
        end_day.value = None
        # printing out the exceptions if they occur
    except Exception as e:
        with output_widget_listimages:
            output_widget_listimages.clear_output()
            print(Fore.RED + "An unknown error occured.")
            print(e)
        
# event listener when user click on button clear fields
clear_fields.on_click(clear_fields_clicked)

In [ ]:
# after the image from the list is selected, the user clicks the button submit
# the event hadler for submit button
def on_submit_clicked(b):
    # using the output widget to print the confirmation
    with output_widget_img:
        output_widget_img.clear_output()
        
        try:
            # access the value of the date from the list that could be accesed globally (the value -1 as we did tranformation)
            day = list_dates[day_select_widget.value-1]
            
            # the way to retrieve the image from the defined collection above is by date
            # in order to do so, the date of start is by default 00:00:00, we are also making the date for the same day but 23:59:59
            # this way we could retrive all images that are made in 24 hours of one day
            day_end = day + 'T23:59:59'
            day_start = ee.Date(day)
            day_end = ee.Date(day_end)
            # filtering the collection to get one image of interest
            selected_images = collection.filterDate(day_start, day_end)
            global one_image #to take the date from one image
            one_image = selected_images.first()
            mosaic = selected_images.mosaic() # mosaic to merge images the same datу, next or previous date already with a gap
            # making the final image global variable to be accesed from inside other fucntions
            global image_final
            # ususlly there doubled images for the same date, so we are selecting only first one
            image_final = mosaic
            print("The image is selected!") # print with widget that image is succesfully selected
        except Exception as e:
            # showing the exception if occurs
            print(e)
            print(Fore.RED + 'Try to select again the number from the list. An error occured!')

# event listener for submit button
submit_sel.on_click(on_submit_clicked)

* **Define the visualization parameters in the panel below**

Firstly, select the band combinations to visulaize. You could blend two options by changing their opacity with a slider under each dropdown window. Furthermore, stretching parameters for the initial image could be modiefied by varying minimum and maximum stretching values. Other parameters are saturation of the image and sensitivity of fire detection.
After all, there is a manual adjustment of the final image and visualization parameters: minimum and maximum values.

In [ ]:
# The next section is visualiaing a series of widgets to define the visualization parameters of the image

# Drop down widgets to choose band combination 1 and 2 for visualizations, that will be blended on the next step
visbands = widgets.Dropdown(description='Select band combination 1:',
                                  options=['Natural Colors', 'Enhanced Natural Colors', 'NIR SWIR Colors 1', 'NIR SWIR Colors 2', 'NIR SWIR Colors 3', 'NIR SWIR Colors 4', 'Natural NIR SWIR Mix', 'False Color', 'Natural False Color',
                                          'Vegetation', 'Pan Band'], value = 'NIR SWIR Colors 2', style=style, layout=widgets.Layout(width='30%'))
visbands_2 = widgets.Dropdown(description='Select band for combination 2:',
                                  options=['Natural Colors', 'Enhanced Natural Colors', 'NIR SWIR Colors 1', 'NIR SWIR Colors 2', 'NIR SWIR Colors 3', 'NIR SWIR Colors 4', 'Natural NIR SWIR Mix', 'False Color', 'Natural False Color',
                                          'Vegetation', 'Pan Band'], value = 'Enhanced Natural Colors', style=style, layout=widgets.Layout(width='30%'))

# Sliders to select the opacity of the layers
opac_1 = widgets.FloatSlider(description = 'Opacity 1:', value = 0, min = 0, max = 1, step = 0.05, layout=widgets.Layout(width='30%'))
opac_2 = widgets.FloatSlider(description = 'Opacity 2:', value = 1, min = 0, max = 1, step = 0.05, layout=widgets.Layout(width='30%'))
# Boxes to make a layout
visbands_widgets = widgets.HBox([visbands, visbands_2])
opac_widget = widgets.HBox([opac_1, opac_2])

# Stretching the image widgets to adjust the lightness
# Saturation widget to make image more or less vivid (0 is monochrome)
# Sensitivity defines the hotspot detection, more hotspots will be detected with higher numbers
stretch_min = widgets.FloatText(description = 'Stretch min:', step = 0.05, value = 0.01, layout=widgets.Layout(width='16%'))
stretch_max = widgets.FloatText(description = 'Stretch max:', width = '20%', step = 0.05, value = 0.9, layout=widgets.Layout(width='16%'))
satur = widgets.FloatText(description = 'Saturation:', width = '20%', step = 0.05, value = 1, layout=widgets.Layout(width='16%'))
sensit = widgets.FloatText(description = 'Fire sensitivity:', width = '20%', step = 0.05, value = 1, layout=widgets.Layout(width='16%'))

# widgets to define burnscar visualization
burnscarHighlight = widgets.FloatText(description = 'Highlight:', step = 0.05, value = 0, layout=widgets.Layout(width='16%'))
thresholdLow =  widgets.FloatText(description = 'Thsd low:', step = 0.05, value = -0.25, layout=widgets.Layout(width='16%'))
thresholdHigh =  widgets.FloatText(description = 'Thsd high:', step = 0.05, value = -0.38, layout=widgets.Layout(width='16%'))
desatBackdrop = widgets.FloatText(description = 'Desat back:', step = 0.05, value = 0.25, layout=widgets.Layout(width='16%'))
darkenBackdrop = widgets.FloatText(description = 'Darken back:', step = 0.05, value = 0.25, layout=widgets.Layout(width='16%'))
burnscar = widgets.HBox([burnscarHighlight, thresholdLow, thresholdHigh, desatBackdrop, darkenBackdrop])

# layout for the widgets
stretch_widget = widgets.HBox([stretch_min, stretch_max, satur, sensit])

# widgets for final manual correction of the images with horizontal box layout
corr_R = widgets.FloatText(value = 0, description = 'Adjust R:', step = 0.05, layout=widgets.Layout(width='16%'))
corr_G = widgets.FloatText(value = 0, description = 'G:', width = '20%', step = 0.05, layout=widgets.Layout(width='16%'))
corr_B = widgets.FloatText(value = 0, description = 'B:', width = '20%', step = 0.05, layout=widgets.Layout(width='16%'))
correction_widget = widgets.HBox([corr_R, corr_G, corr_B])

# widget to choose between clipping the image to fit the AOI or not 
cropping_widget = widgets.RadioButtons(options=['Crop ROI', 'Show full image'], description='Crop area:', value = 'Show full image', disabled=False)

# the button to visualize the image using the defined parameters, and the button to clear the map from the layers
vis = widgets.Button(description='Visualize', button_style='primary', tooltip='Click to visualize', style=style)
clear_map = widgets.Button(description='Clear map', button_style='warning', tooltip='Click to clear', style=style)
export_image = widgets.Button(description='Export image', tooltip='Click to export', style=style)
export_image.style.button_color = 'turquoise'

fc = FileChooser('')
fc.show_only_dirs = True

# layout for buttons and final layout of the panel
widgets_mapping = widgets.VBox([fc, widgets.HBox([vis, clear_map, export_image])])
visual_widget = widgets.VBox([widgets.Label('Visualization selection'), visbands_widgets, opac_widget, 
                              widgets.Label('Image and fire detection adjustment'), stretch_widget, correction_widget,
                              widgets.Label('Burnscar highlight'), burnscar,
                              widgets.Label('Cropping and visalization'), cropping_widget, widgets_mapping])
display(visual_widget)

In [ ]:
# the fucntion to clear map and delete all layers exept the basemap
def clear_map_clicked(b):
        m.clear_layers()
        m.add_basemap('TERRAIN')
        
# on_click is an event listener that call a function when clicked on the button
clear_map.on_click(clear_map_clicked)

In [ ]:
def on_export_clicked(b):
    with output_widget:
        output_widget.clear_output()
        print('Exporting...')
        Map.default_style = {'cursor': 'wait'}
        
        try: 
            dir_file = fc.value
            dir_file.replace(r'\\', '/') # adjust for python backslahes
            complete_name = os.path.join(dir_file, export_date + '.tif') # full name of the image with path
            print(complete_name)
            geemap.ee_export_image(final_image_export, filename=complete_name, file_per_band=False, scale=20)
            
            print('Export completed!')
            output_widget.clear_output()
        except Exception as e:
            print(e)
            print(Fore.RED + 'Try again! An error occured')
            
export_image.on_click(on_export_clicked)

In [ ]:
# the following chunck with exact visualization script is based on the script of Pierre Markuse for Sentinel Hub and adapted for use with Earth Engine
# https://pierre-markuse.net/2018/04/30/visualizing-wildfires-burn-scars-sentinel-hub-eo-browser/

# function to make a visualization when on submit clicked
def visualize(b):
    
    # print with widget on the map that the process is ongoing
    with output_widget:
        output_widget.clear_output()
        print('Visualazing...')
        Map.default_style = {'cursor': 'wait'}
        
        try:
            # the sentinel image should be divided by 10000 to have values of reflectance
            image = image_final.divide(10000)
            
            # image stretching based on provided parameters
            def stretch(image, min, max):
                return (image.subtract(min)).divide(max-min)
            
            # define the different visualization for fire/lava, 3 levels based on intensity: red, orange and yellow
            Fire1OVL = [stretch((image.select('B4').multiply(2.1).add(image.select('B12').multiply(0.5))), 0.01, 0.99).add(1.1), 
                    stretch((image.select('B3').multiply(2.2).add(image.select('B8').multiply(0.5))), 0.01, 0.99), 
                    stretch(image.select('B2').multiply(2.1), 0.01, 0.99)]
            Fire2OVL = [stretch((image.select('B4').multiply(2.1).add(image.select('B12').multiply(0.5))), 0.01, 0.99).add(1.1), 
                    stretch((image.select('B3').multiply(2.2).add(image.select('B8').multiply(0.5))), 0.01, 0.99).add(0.25), 
                    stretch(image.select('B2').multiply(2.1), 0.01, 0.99)]
            Fire3OVL = [stretch((image.select('B4').multiply(2.1).add(image.select('B12').multiply(0.5))), 0.01, 0.99).add(1.1), 
                    stretch((image.select('B3').multiply(2.2).add(image.select('B8').multiply(0.5))), 0.01, 0.99).add(0.5), 
                    stretch(image.select('B2').multiply(2.1), 0.01, 0.99)]
            
            # make an images from fire layers
            fire_layers = [ee.Image(Fire1OVL), ee.Image(Fire2OVL), ee.Image(Fire3OVL)]
            
            # the function to select a band combination of interest (chosen by user)
            def combo_selection(input, image):
                if input == 'Natural Colors':
                    layer = [image.select('B4').multiply(2.9), image.select('B3').multiply(3.1), image.select('B2').multiply(3.0)] #NatruralColors
                elif input == 'Enhanced Natural Colors':
                    layer = [(image.select('B4').multiply(2.8)).add(image.select('B5').multiply(0.1)), (image.select('B3').multiply(2.8)).add(image.select('B8').multiply(0.15)),
                                 image.select('B2').multiply(2.8)] #Enhanced Natural Colors
                elif input == 'NIR SWIR Colors 1':
                    layer = [image.select('B12').multiply(2.6), image.select('B8').multiply(1.9), image.select('B2').multiply(2.7)] #NIRSWIRColors1
                elif input == 'NIR SWIR Colors 2':
                    layer = [image.select('B12').multiply(2.4), image.select('B8A').multiply(1.7), image.select('B5').multiply(2.2)] #NIRSWIRColors2
                elif input == 'NIR SWIR Colors 3':
                    layer = [((image.select('B12').add(image.select('B11'))).multiply(0.5).divide(4)).divide(image.select('B7')), image.select('B8A').multiply(0.8), image.select('B7')] #NIRSWIRColors3
                elif input == 'NIR SWIR Colors 4':
                    layer = [image.select('B12').multiply(2.0), image.select('B11').multiply(1.1), image.select('B8').multiply(1.6)] #NIRSWIRColors4
                elif input == 'Natural NIR SWIR Mix':
                    layer = [(image.select('B4').multiply(2.1)).add(image.select('B12').multiply(0.5)), (image.select('B3').multiply(2.2)).add(image.select('B8').multiply(0.5)),
                             image.select('B2').multiply(3.0)] #NaturalNIRSWIRMix
                elif input == 'False Color':
                    layer = [image.select('B8').multiply(2), image.select('B4').multiply(2), image.select('B3').multiply(2)] #FalseColor
                elif input == 'Natural False Color':
                    layer = [image.select('B12').multiply(2.6), image.select('B11').multiply(2), image.select('B4').multiply(2.7)] #NatFalseColor
                elif input == 'Vegetation':
                    layer = [image.select('B11').multiply(2.4), image.select('B8A').multiply(2), image.select('B4').multiply(2.9)] #Vegetation
                elif input == 'Pan Band':
                    layer = [image.select('B8'), image.select('B8'), image.select('B8')] #PanBand
                return layer
            
            # defining layers from band combinations defined by user in a widget
            layer1 = combo_selection(visbands.value, image)
            layer2 = combo_selection(visbands_2.value, image)
            # getting opacity value from a widget
            layer1Amount = opac_1.value
            layer2Amount = opac_2.value
            
            #getting parameters from widgets
            stretchMin = stretch_min.value
            stretchMax = stretch_max.value
            saturation = satur.value
            fireSensitivity = sensit.value
            manualCorrection = [corr_R.value, corr_G.value, corr_B.value]
            
            # belnding two images by opacity
            # bands are given in arrays, and opacity
            def blend(bArr1, bArr2, opa1, opa2):
                result_list =[]
                # loop through 2 arrays in the same time
                for (a_img,b_img) in zip(bArr1, bArr2):
                    result_list.append(a_img.divide(opa1).add(b_img.divide(opa2)))
                return result_list
            
            # get no fire layer 
            
            global noFire
            noFire = blend(layer1, layer2, layer1Amount, layer2Amount)
            

            burnscarHl = burnscarHighlight.value
            burnscarThresholdLow = thresholdLow.value
            burnscarThresholdHigh = thresholdHigh.value
            burnscarDesaturateBackdrop = desatBackdrop.value
            burnscarDarkenBackdrop = darkenBackdrop.value
            
            global stretch_max1, stretch_max1init, saturat, saturatinit
            stretch_max1 = ee.Image(stretchMax)
            saturat = ee.Image(saturation)
            
            def highlightBurnscar(val, oLow, oHigh, deSat, darken):
                if val > 0:
                    global stretch_max1, saturat, noFire
                    noFire_init = noFire
                    
                    #for the noFire background values
                    noFire[0] = noFire[0].where((image.select('B12').add(image.select('B11'))).gt(0.05), noFire[0].add(0.15+val))
                    noFire[1] = noFire[1].where((image.select('B12').add(image.select('B11'))).gt(0.05), noFire[1].add(0.15+val))
                    
                    noFire[0] = noFire[0].where(((image.select('B12').add(image.select('B11'))).gt(0.05)).
                                                      And((image.select('B8A').subtract(image.select('B12'))).
                                                         divide(image.select('B8A').add(image.select('B12'))).lte(oHigh)), noFire[0].add(0.2+val))
                    
                    noFire[1] = noFire[1].where(((image.select('B12').add(image.select('B11'))).gt(0.05)).
                                                      And((image.select('B8A').subtract(image.select('B12'))).
                                                         divide(image.select('B8A').add(image.select('B12'))).lte(oHigh)), noFire[1].add(0.05+val))
                    
                    saturat = saturat.where(((image.select('B12').add(image.select('B11'))).gt(0.05)).
                                                      And((image.select('B8A').subtract(image.select('B12'))).
                                                         divide(image.select('B8A').add(image.select('B12'))).gt(oLow)), saturat.subtract(deSat))
                    stretch_max1 = stretch_max1.where(((image.select('B12').add(image.select('B11'))).gt(0.05)).
                                                      And((image.select('B8A').subtract(image.select('B12'))).
                                                         divide(image.select('B8A').add(image.select('B12'))).gt(oLow)), stretch_max1.add(darken))
                    
                    #for the noFire 
                    noFire[0] = noFire_init[0].where(((image.select('B12').add(image.select('B11'))).gt(0.05)).
                                                      And((image.select('B8A').subtract(image.select('B12'))).
                                                         divide(image.select('B8A').add(image.select('B12'))).gt(oLow)), noFire_init[0])
                    noFire[1] = noFire_init[1].where(((image.select('B12').add(image.select('B11'))).gt(0.05)).
                                                      And((image.select('B8A').subtract(image.select('B12'))).
                                                         divide(image.select('B8A').add(image.select('B12'))).gt(oLow)), noFire_init[1])  

                    
                    global stretchMax, saturation            
                    stretchMax = stretch_max1
                    saturation = saturat
            
            
            # enhacement of visualization
            def satEnh(rgbArr):
                avg = functools.reduce(lambda a, b: a.add(b), rgbArr).divide(len(rgbArr))
                result = [(avg.multiply(1 - saturation)).add(img.multiply(saturation)) for img in rgbArr]
                return result 
            
            # application of the enhancement of the blended image
            def applyEnh(bArr):
                highlightBurnscar(burnscarHl, burnscarThresholdLow, burnscarThresholdHigh, burnscarDesaturateBackdrop, burnscarDarkenBackdrop)
                return satEnh([stretch(bArr[0], stretchMin, stretchMax), stretch(bArr[1], stretchMin, stretchMax), stretch(bArr[2], stretchMin, stretchMax)])
            
            # manual correction if defined
            def correction(manual_num, arrRGB):
                return [arrRGB[0].add(manual_num[0]), arrRGB[1].add(manual_num[1]), arrRGB[2].add(manual_num[2])]
            
            # applcation of enhancement and correction for the no fire image and final renaming of the bands
            noFire_final = applyEnh(noFire)
            noFire_final = correction(manualCorrection, noFire_final)
            noFire_final = ee.Image(noFire_final).rename(['R_new', 'G_new', 'B_new'])
            
            # adding swir difference and choosing conditionally to assign fire lvayers value depending on SWIR difference value
            img_exp = image.select('B11').add(image.select('B12')).rename('SWIR_dif') 
            noFire_final = noFire_final.where(img_exp.select("SWIR_dif").gt(1/fireSensitivity), fire_layers[0])
            noFire_final = noFire_final.where(img_exp.select("SWIR_dif").gt(1.5/fireSensitivity), fire_layers[1])
            noFire_final = noFire_final.where(img_exp.select("SWIR_dif").gt(2/fireSensitivity), fire_layers[2])
                 
            # visual parameters and date, date to put it in the image name
            visParams = {"min": stretch_min.value, "max": stretch_max.value, "bands": ["R_new", "G_new", "B_new"]}
            date = ee.Date(one_image.get('system:time_start')).format('YYYY-MM-dd')
            
            global final_image_export, export_date
            final_image_export = noFire_final.clip(roi).unmask()
            export_date = date.getInfo()
            
            # clip the image by boundary and visualize or just visualize th whole image
            if cropping_widget.value == 'Crop ROI':   
                m.addLayer(noFire_final.clip(roi), visParams,  visbands.value + ' ' + visbands_2.value + '; ' + date.getInfo())
            else:
                m.addLayer(noFire_final, visParams, visbands.value + ' ' + visbands_2.value + '; ' + date.getInfo())
            output_widget.clear_output()
            
        except Exception as e:
            print(e)
            print(Fore.RED + 'Try again! An error occured')
            
vis.on_click(visualize)

In [ ]:
# the function to retrieve the image collection through earth engine
def collection_retr(start, finish, roi):
    collection = "COPERNICUS/S2" 
    s2_coll = (ee.ImageCollection(collection)
        .filterDate(start, finish)
        .filterBounds(roi)) # filtering with dates provided to a fucntion and boundaries
    return s2_coll